In [2]:
import numpy as np
import tensorflow as tf
import cv2

/Users/gavinxue/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
cap = cv2.VideoCapture(1)

# Display frame that is inputted to model and frame with detections
cv2.namedWindow("Detection frame", cv2.WINDOW_AUTOSIZE)

# Threshold value [0.0, 1.0] 
detection_threshold = 0.1

# Conversion vector from bounding box coordinate to display frame coordinate
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
bbox_coord_conversion = np.array([frame_height,frame_width,frame_height,frame_width])

saved_model_dir = './saved_model_face/'

with tf.compat.v1.Session() as sess:
    # Load model from saved_model directory
    metagraph = tf.compat.v1.saved_model.load(sess, [tf.compat.v1.saved_model.tag_constants.SERVING], saved_model_dir)

    # Get input and output tensors to dictionary format
    input_dict = dict(metagraph.signature_def['serving_default'].inputs)
    output_dict = dict(metagraph.signature_def['serving_default'].outputs)

    # Get input and output tensors
    input_tensor = sess.graph.get_tensor_by_name(input_dict['inputs'].name)
    bounding_boxes = sess.graph.get_tensor_by_name(output_dict['detection_boxes'].name)
    box_scores = sess.graph.get_tensor_by_name(output_dict['detection_scores'].name)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Retrieve bounding boxes and their scores, no class since only 1 class: face
        [boxes, scores] = sess.run([bounding_boxes, box_scores],
                                   feed_dict={input_tensor: np.expand_dims(frame, axis=0)})
        
        # Only select bounding boxes above threshold value
        detections = boxes[scores > detection_threshold]

        # Perform bounding box conversion and display box on frame
        for face in detections:
            bbox_coord = (face * bbox_coord_conversion).astype(int)
            cv2.rectangle(frame, (bbox_coord[1], bbox_coord[0]), (bbox_coord[3], bbox_coord[2]), (0, 255, 0), 2)

        cv2.imshow("Detection frame", frame)

        # Exit program with 'q' key
        key = cv2.waitKey(1) & 0xff
        if key == ord('q'):
            cv2.destroyAllWindows()
            break

2024-01-14 00:47:14.302 Python[82216:20823162] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Instructions for updating:
Use `tf.saved_model.load` instead.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.


2024-01-14 00:47:16.005390: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-01-14 00:47:16.655 Python[82216:20823162] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


: 